# Calculate geostrophic velocities at all depths

In [1]:
import os
import cosima_cookbook as cc
from dask.distributed import Client

import numpy as np
import xarray as xr

from oceanpy import define_grid

from gsw import f, p_from_z, grav

from numbers import Number
from scipy.ndimage import uniform_filter#, gaussian_filter

In [2]:
outdir = os.path.join(os.sep, 'g', 'data', 'v45', 'jm6603', 'checkouts', 'phd', 'src', 'cosima', '02_manuscript', 'output')
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [3]:
def to_netcdf(ds, file_name):

    valid_types = (str, Number, np.ndarray, np.number, list, tuple)
    try:
        ds.to_netcdf(file_name)
    except TypeError as e:
        print(e.__class__.__name__, e)
        for variable in ds.variables.values():
            for k, v in variable.attrs.items():
                if not isinstance(v, valid_types) or isinstance(v, bool):
                    variable.attrs[k] = str(v)
        ds.to_netcdf(file_name)

In [4]:
def geostrophic_velocity(ds, grid, sea_level='sea_level', stream_func='deltaD', gravity='gu', coriolis='f', delta_names=('dx', 'dy')):

    '''
    calculate geostrophic velocity from level of 'known' motion (e.g. surface)
    '''
    
    # surface geostrophic velocity
    detadx = grid.interp(grid.diff(ds[sea_level], 'X', boundary='extend'), 'Y', boundary='extend') / ds[delta_names[0]]
    detady = grid.interp(grid.diff(ds[sea_level], 'Y', boundary='extend'), 'X', boundary='extend') / ds[delta_names[1]]

    ds['ug_s']= - (ds[gravity].isel(st_ocean=0) / ds[coriolis]) * detady
    ds['vg_s'] = (ds[gravity].isel(st_ocean=0) / ds[coriolis]) * detadx

    ds['ug_s'].name = 'ug_s'
    ds['ug_s'].attrs['standard_name'] = 'surface_geostrophic_eastward_sea_water_velocity'
    ds['ug_s'].attrs['long_name'] = r'$u_g,s$'
    ds['ug_s'].attrs['units'] = r'$\mathrm{ms}^{-1}$'
    
    ds['vg_s'].name = 'vg_s'
    ds['vg_s'].attrs['standard_name'] = 'surface_geostrophic_northward_sea_water_velocity'
    ds['vg_s'].attrs['long_name'] = r'$v_g,s$'
    ds['vg_s'].attrs['units'] = r'$\mathrm{ms}^{-1}$'
    
    if not stream_func == None:
        ds['ug'] = ds['ug_s'] + grid.interp(grid.diff(ds[stream_func], 'Y', boundary='extend'), 'X', boundary='extend')  / (ds[delta_names[1]] * ds[coriolis])
        ds['vg'] = ds['vg_s'] - grid.interp(grid.diff(ds[stream_func], 'X', boundary='extend'), 'Y', boundary='extend')  / (ds[delta_names[0]] * ds[coriolis])

        ds['ug'].name = 'ug'
        ds['ug'].attrs['standard_name'] = 'geostrophic_eastward_sea_water_velocity'
        ds['ug'].attrs['long_name'] = r'$u_g$'
        ds['ug'].attrs['units'] = r'$\mathrm{ms}^{-1}$'

        ds['vg'].name = 'vg'
        ds['vg'].attrs['standard_name'] = 'geostrophic_northward_sea_water_velocity'
        ds['vg'].attrs['long_name'] = r'$v_g$'
        ds['vg'].attrs['unidatats'] = r'$\mathrm{ms}^{-1}$'
    
    return ds

## Load data

In [5]:
session = cc.database.create_session()
expt = '01deg_jra55v140_iaf'

In [6]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/39837/status,
Dashboard: /proxy/39837/status,Workers: 7
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39881,Workers: 7
Dashboard: /proxy/39837/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:45671,Total threads: 2
Dashboard: /proxy/36333/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:33611,


In [7]:
# data output frequency
freq = '1 daily'

# time limits of dataset
start, end = '1997-03-01', '1997-05-30'

# location limits of dataset
lon_lim = slice(-225.2, -210.8)
lat_lim = slice(-53.7, -46.3)

meander_period = slice('1997-02-15', '1997-05-31')
monthly_period = slice('1997-04-01', '1997-04-30')

### Load and select coordinates

In [8]:
dxt = cc.querying.getvar(expt=expt, variable='dxt', session=session, frequency='static', n=1)
dyt = cc.querying.getvar(expt=expt, variable='dyt', session=session, frequency='static', n=1)
dzt = cc.querying.getvar(expt=expt, variable='dzt', session=session, frequency='1 monthly', n=1)

dxu = cc.querying.getvar(expt=expt, variable='dxu', session=session, frequency='static', n=1)
dyu = cc.querying.getvar(expt=expt, variable='dyu', session=session, frequency='static', n=1)

area_t = cc.querying.getvar(expt=expt, variable='area_t', session=session, frequency='static', n=1)
area_u = cc.querying.getvar(expt=expt, variable='area_u', session=session, frequency='static', n=1)

kmu = cc.querying.getvar(expt=expt, variable='kmu', session=session, frequency='static', n=1)
kmt = cc.querying.getvar(expt=expt, variable='kmt', session=session, frequency='static', n=1)

geolat_t = cc.querying.getvar(expt, variable='geolat_t', session=session, n=1)
geolon_t = cc.querying.getvar(expt, variable='geolon_t', session=session, n=1)

In [9]:
dxt_lim = dxt.sel(xt_ocean=lon_lim, yt_ocean=lat_lim)
dyt_lim = dyt.sel(xt_ocean=lon_lim, yt_ocean=lat_lim)
dzt_lim = dzt.sel(xt_ocean=lon_lim, yt_ocean=lat_lim).isel(time=1)
dzt_lim.name = 'dst'

dxu_lim = dxu.sel(xu_ocean=lon_lim, yu_ocean=lat_lim)
dyu_lim = dyu.sel(xu_ocean=lon_lim, yu_ocean=lat_lim)

areat_lim = area_t.sel(xt_ocean=lon_lim, yt_ocean=lat_lim)
areau_lim = area_u.sel(xu_ocean=lon_lim, yu_ocean=lat_lim)

kmu_lim = kmu.sel(xu_ocean=lon_lim, yu_ocean=lat_lim)
kmt_lim = kmt.sel(xt_ocean=lon_lim, yt_ocean=lat_lim)

lat_t = geolat_t.sel(xt_ocean=lon_lim, yt_ocean=lat_lim)
lon_t = geolon_t.sel(xt_ocean=lon_lim, yt_ocean=lat_lim)

### Load and select variables

In [10]:
# velocities
sl = cc.querying.getvar(expt=expt, variable='sea_level', session=session, frequency=freq, start_time=start, end_time=end)
u = cc.querying.getvar(expt=expt, variable='u', session=session, frequency=freq, start_time=start, end_time=end)
v = cc.querying.getvar(expt=expt, variable='v', session=session, frequency=freq, start_time=start, end_time=end)
wt = cc.querying.getvar(expt=expt, variable='wt', session=session, frequency=freq, start_time=start, end_time=end)

In [11]:
sl_lim = sl.sel(xt_ocean=lon_lim, yt_ocean=lat_lim)
u_lim = u.sel(xu_ocean=lon_lim, yu_ocean=lat_lim)
v_lim = v.sel(xu_ocean=lon_lim, yu_ocean=lat_lim)
wt_lim = wt.sel(xt_ocean=lon_lim, yt_ocean=lat_lim)

## Constants

In [12]:
# g = 9.81
rho_0 = 1036 # kg/m^3
p_ref = 0 #1500
p_mld = 200
p_int = 2500

## Define grid

In [13]:
# define coordinates
coords = {'xt_ocean': None, 'yt_ocean': None, 'st_ocean': None, 'xu_ocean': 0.5, 'yu_ocean': 0.5, 'sw_ocean': -0.5}
distances=('dxt', 'dyt', 'dst', 'dxu', 'dyu')
areas=('area_u', 'area_t')
dims=('X', 'Y', 'S')

coordinates = xr.merge([dxt_lim, dyt_lim, dzt_lim, dxu_lim, dyu_lim, areat_lim, areau_lim])
vel = xr.merge([coordinates, u_lim, v_lim, wt_lim])

# define grid
grid = define_grid(vel, dims, coords, distances, areas, periodic=False)

In [14]:
file_name = os.path.join(outdir, 'vel.nc')
if not os.path.exists(file_name):
    vel_sel = vel.sel(time=slice(start, end))
    to_netcdf(vel_sel, file_name)

TypeError Invalid value for attr 'c_grid_axis_shift': None. For serialization to netCDF files, its value must be of one of the following types: str, Number, ndarray, number, list, tuple


## Calculate geostrophic wind

In [15]:
file_name = os.path.join(outdir, 'geos-vel.nc')
if os.path.exists(file_name):
    geos_vel = xr.open_dataset(file_name)
else:
    # Load hydrographic dataset
    ds = xr.open_dataset(os.path.join(outdir, 'hydro.nc'))
    hydro = ds.sel(time=monthly_period)
    
    # Pressure coordinate
    lat_t_3d = lat_t.broadcast_like(coordinates.dst)
    z_3d = (-coordinates.st_ocean).broadcast_like(coordinates.dst)
    p_3d = xr.apply_ufunc(p_from_z, z_3d, lat_t_3d, dask='parallelized', output_dtypes=[z_3d.dtype])
    p_3d = p_3d.compute()
    
    # Gravitational acceleration
    g = xr.apply_ufunc(grav, lat_t_3d, p_3d, dask='parallelized', output_dtypes=[p_3d.dtype])
    g.name = 'g'
    g.attrs = {'standard_name': 'sea_water_pressure', 'units':r'$\textrm{ms}^{-2}$', 'long_name': 'gravitational acceleration'}
    
    # Coriolis parameter
    fcor,_ = xr.broadcast(f(coordinates.yu_ocean), coordinates.xu_ocean)
    fcor.name = 'f'
    fcor.attrs = {'standard_name': 'coriolis_parameter', 'units': r'$\textrm{s}^{-1}$', 'long_name': 'Coriolis parameter'}
    
    gu = grid.interp(grid.interp(g, 'X', boundary='extend'), 'Y', boundary='extend')
    gu.name = 'gu'
    # D = deltaD.sel(st_ocean=p_int, method='nearest') / g.sel(st_ocean=p_int, method='nearest')
    
    # Dynamic height
    D = (hydro.deltaD / g)
    D.name = 'D'
    D.attrs = {'units': 'm', 'long_name': 'Dynamic height'}
    
    deltaD_grd = xr.merge([
        coordinates, hydro.deltaD, D, g, gu, fcor,
        sl_lim.sel(time=monthly_period),
        u_lim.sel(time=monthly_period),
        v_lim.sel(time=monthly_period)])
    
    # # g = gsw.grav(lat_t_3d, pressure)
    # # D = deltaD.sel(st_ocean=p_int, method='nearest') / g.sel(st_ocean=p_int, method='nearest')
    
    # Geostrophic velocity dataset
    geos_vel = geostrophic_velocity(deltaD_grd, grid, delta_names=('dxu', 'dyu'))
    
    # Residual velocities after substraction of geostrophic velocities
    geos_vel['ures'] = geos_vel.u - geos_vel.ug
    geos_vel['vres'] = geos_vel.v - geos_vel.vg
    
    geos_vel['Vg_s'] = (geos_vel.ug_s**2 + geos_vel.vg_s**2) ** (1/2)
    geos_vel['Vg'] = (geos_vel.ug**2 + geos_vel.vg**2) ** (1/2)
    geos_vel['Vres'] = (geos_vel.ures**2 + geos_vel.vres**2)**(1/2)

    # Save geostrophic velocity dataset to file
    # geos_vel = geos_vel.drop_vars(['dxt', 'dyt', 'dst', 'area_t', 'area_u'])
    to_netcdf(geos_vel, file_name)

TypeError Invalid value for attr 'time_bounds': <xarray.DataArray 'time_bounds' (time: 3, nv: 2)>
dask.array<open_dataset-58c27d34cd3c9dac8f214dabff64eef4time_bounds, shape=(3, 2), dtype=timedelta64[ns], chunksize=(1, 2), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1958-01-16T12:00:00 ... 1958-03-16T12:00:00
  * nv       (nv) float64 1.0 2.0
Attributes:
    long_name:  time axis boundaries
    calendar:   GREGORIAN. For serialization to netCDF files, its value must be of one of the following types: str, Number, ndarray, number, list, tuple
